In [3]:
# This is the code for SI 671 Kaggle competition 1

In [4]:
import pandas as pd
import numpy as np


In [5]:
kaggle_train = pd.read_csv('si670_kaggle1_train.csv', index_col=2)
kaggle_train.head()

,text,label
id,,
0,White girls very rarely date Asian men. Even i...,1
1,I am a 23 year old male Indian American male. ...,1
2,"Take three people, Persons A, B, and C. They l...",1
3,(A) Work part-time in high school; Then go to ...,1
4,When police introduce a new form of speed prev...,1


In [6]:
kaggle_validation = pd.read_csv('si670_kaggle1_validation.csv', index_col=2)
kaggle_validation.head()

,text,label
id,,
319071,"In much the same way as drugs, prostitution, g...",1
319072,A friend and I were talking about the current ...,1
319073,While I believe this is primarily an American ...,1
319074,The basis of this viewpoint is that a system t...,1
319075,"Ah, like it says in ""Dust In The Wind."" Every ...",1


In [7]:
kaggle_test = pd.read_csv('test.csv', index_col=1)
kaggle_test.tail(1)

,id
text,
"The effects on miRNAs caused by HIVSIV infection are illustrated by changes in their cellular expression throughout the course of disease, and in different patient populations. Our data demonstrate that levels of primary transcripts and mature miRs-29a, -29b, -9 and -146a are modulated by SIV infection. We show that the SIV 3' UTR contains functional miRNA response elements (MREs) for all four miRNAs. Notably, these miRNAs regulate virus production and viral RNA levels in macrophages, the primary cells infected in the CNS that drive inflammation leading to HIV-associated neurocognitive disorders. This report may aid in identification miRNAs that target viral RNAs and HIVSIV specifically, as well as in identification of miRNAs that may be targets of new therapies to treat HIV.",436605


In [10]:
# first, a baseline model with TF-IDF and logistic regression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

pipe = Pipeline([
    ('vect', TfidfVectorizer(analyzer='char_wb', stop_words='english', ngram_range=(3, 5), min_df=2)),
    ('logreg', LogisticRegression())
])

X_train, X_test, y_train, y_test = train_test_split(
    kaggle_train['text'], kaggle_train['label'], test_size=0.2, random_state=42)

param_grid = [
    {
        'vect__analyzer': ['word', 'char_wb'],
        'vect__ngram_range': [(1, 1), (1, 2), (2, 3), (3, 5)],
        'vect__min_df': [1, 2],
        'logreg__C': [0.01, 0.1, 1, 10],
        'logreg__penalty': ['l1', 'l2'],
        'logreg__solver': ['lbfgs']
    }
]

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=3,
    scoring="f1",
    n_jobs=-1
)
grid.fit(X_train, y_train)
print("Best parameters:", grid.best_params_)
print("Best CV score:", grid.best_score_)

classification_report(y_test, grid.predict(X_test))


KeyboardInterrupt: 